# Crise du sida Berlin
Ce notebook Jupiter contient un **[tableau](#Tableau)** de tous les lieux visités pendant l'atelier topographique sur la crise du sida à Berlin dans les années 80 (*appel à participation [DE](https://www.sciencespo.fr/histoire/fr/node/89436.html) | [FR](https://cmb.hu-berlin.de/fr/agenda/evenement/topografischer-geschichtsworkshop-fuer-junge-forscherinnen)*), et une **[carte dynamique](#Carte)** générée à partir de ce tableau. 

L'atelier s'est tenu au Marc Bloch Zentrum du 13 au 18 mai 2024 et a entre autres donné lieu à un interview de l'une des organisatrices se trouve [ici](https://www.sciencespo.fr/fr/actualites/la-crise-du-sida-a-berlin-parcourir-une-ville-pour-comprendre-son-histoire/), la création d'une [librairie Zotero](https://www.zotero.org/groups/5537257/aidskrisebln2024/library) et d'un [répertoire de travail commun](https://sharedocs.huma-num.fr/wl/?id=Su3ebB9UxlLW3jueknVkCwTB4B0ckciH), afin d'archiver ce travail sur l'archive et de le poursuivre plus tard. 

## Tableau 

Ce tableau est éditable en .xlsx sur le [répertoire ](https://sharedocs.huma-num.fr/wl/?id=on6pmqexVsfysrKr8E8QwjktVP7gaABW&fmode=edit). 

In [109]:
import pandas as pd
import folium

df_raw = pd.read_csv("synth-lieux.csv", encoding='utf-8', sep=';')
df = df_raw[df_raw['Visited'] == "oui"]
df

,ID,Place,Visited,Adresse,AddrPlain,Type,Now,Timeline,Community,Keywords,East or west,Date,Pics,Unnamed: 13,Unnamed: 14
0,6707Fr92,Centre Marc Bloch,oui,"52.5107745,13.3896707","Friedrichstraße 177-191, 10117 Berlin, Allemagne",Research Center,Centre Marc Bloch,1992,"Researchers, queer activists","Academic reflexivity, community, long-term tra...",East,13.05,"<iframe src=""https://www.google.com/maps/embed...",NaN,NaN
1,9707Ad80,Chez Michel,oui,"52.5025988,13.4199707","Adalbertstraße 83, 10997 Berlin, Allemagne",Restaurant,Chez Michel,1980,French,Taking care of primary body needs,West,13.05,"<iframe src=""https://www.google.com/maps/embed...",NaN,NaN
2,0462Gr39,Yorkstrasse,oui,"52.4916585,13.3660462","Großgörschenstraße 30, 10827 Berlin, Allemagne",Train station,Yorkstrasse,"1981, 1903, 1939","Lesbians, gays, queers","Meeting, circulation, community, togetherness,...",West,14.05,"<iframe src=""https://www.google.com/maps/embed...",NaN,NaN
3,6303Ku99,Les Migras,oui,"52.4919769,13.3646303","Kulmer Str. 20A, 10783 Berlin, Allemagne",Community center,Les Migras,1999,"Lesbians, migrants","Counceling, advocating, community, emotional s...",West,14.05,no street view available,NaN,NaN
4,6303Ku81,Lesbenberatung,oui,"52.4919769,13.3646303","Kulmer Str. 20A, 10783 Berlin, Allemagne",Community center,Lesbenberatung,1981,Lesbians,"Counceling, advocating, community, emotional s...",West,14.05,no street view available,NaN,NaN
5,6303Ku84,Siegesaüle,oui,"52.4919769,13.3646303","Kulmer Str. 20A, 10783 Berlin, Allemagne",Magazine,Siegesaüle,1984,Gays,"Narrating lives, celebrating, meeting, love, s...",West,14.05,no street view available,NaN,NaN
6,8488Po86,Begine - Frauenkneipe und Kultur,oui,"52.4967592,13.3618488","Potsdamer Str. 139, 10783 Berlin, Allemagne",Squat,Begine - Frauenkneipe und Kultur,1986,Lesbians,"Alternative way of life, alternative environme...",West,14.05,"<iframe src=""https://www.google.com/maps/embed...",NaN,NaN
7,3086Yo75,Blocksberg,oui,"52.4932746887207,13.36770248413086","Yorckstraße 47-48, 10783 Berlin, Allemagne",Bar,Fahrschule EinsZweiDrive,1975,Lesbians,"sex, dating, togetherness, knowledge sharing, ...",West,14.05,"<iframe src=""https://www.google.com/maps/embed...",NaN,NaN
8,3082Bü20,Kaiserhof,oui,"52.4965733,13.3653082","Bülowstraße 36, 10783 Berlin, Allemagne",Party,Private home,1920,"Lebians, trans persons, gays","sex, dating, togetherness, alternative tempora...",West,14.05,"<iframe src=""https://www.google.com/maps/embed...",NaN,NaN
9,3834No33,Isherwood,oui,"52.4977986,13.3513834","Nollendorfstraße 17, 10777 Berlin, Allemagne",Individual (writer),Private home,1929-1933,Gays,"Narrating lives, creation, art, long-term tran...",West,14.05,"<iframe src=""https://www.google.com/maps/embed...",NaN,NaN


## Carte

In [110]:
coords= df["Adresse"].tolist()
places = df["Place"].tolist()
typologies = df["Type"].tolist()
communities = df["Community"].tolist()
description = []
images = df["Pics"].tolist()

styling="""<style="color: rgba(251, 238, 230, 1);">"""

# coordonnées 
cleaned_coords = [(lat, lon) for coord in coords if isinstance(coord, str) and ',' in coord for lat, lon in [coord.split(',')]]

coords_with_places = [(coord[0], coord[1], place, typology, community, image) for coord, place, typology, community, image in zip(cleaned_coords, places, typologies, communities, images)]

# Create a map centered at the first coordinate
map_center = (float(coords_with_places[0][0]), float(coords_with_places[0][1]))
my_map = folium.Map(location=map_center, zoom_start=13)

styling_h1 = """<span style="color: rgb(251,153,142); font-weight: 900; font-size: 150%;">"""
styling_h2 = """<span style="color: rgb(253,193,186); font-weight: 600; font-size: 120%;">"""

# Add markers for each coordinate
for loc in coords_with_places:
    lat, lon, place, typologies, communities, images = loc
    folium.Marker(
        location=(float(lat), float(lon)),
        popup=f"""{styling_h1}{place}</span><br>
        {styling_h2}Type</span> {typologies} <br>
        {styling_h2}Communities</span> {communities} <br>
        {styling_h2}Description</span> <br>
        {styling_h2}Street view</span> {images}
        """
    ).add_to(my_map)

# Save the map to an HTML file
my_map
